### 1. telechargement de données

### Base Calling:

Sur le node26: /scratch/ananas

### 2. nettoyage desdonnées

### mapping sur genome ananas:

sur le node30: /scratch/ananas

transformation des bam en reads

In [ ]:
# reads de l'ananas exclusivement
samtools view -@ 4 -bh -F 4 readsONananas.sam > readsONananas_mapped.sam
- 3780387 reads

# reads sans ananas
samtools view -@ 4 -bh -f 4 readsONananas.sam > readsONananas_unmapped.sam
- 1147744 reads

In [ ]:
#on garde quelques reads ananas pour remettre dans le jeu de données plus tard:

module load bioinfo/seqtk/1.3-r106
seqtk sample -s100 readsONananas_mapped.fastq 500000 > sub_readsananas.fastq

### mapping sur bdd fungi:

In [ ]:
#récupération de la banque fungi
wget -r --no-parent -A *genomic.fna.gz ftp://ftp.ncbi.nlm.nih.gov/refseq/release/fungi/
cat *genomic.fna.gz > fungi.genomic.fna.gz

In [ ]:
#mapping des reads qui n'ont pas mappé sur l'ananas sur la banque fungi
minimap2 -ax map-ont --split-prefix=tmp banque/fungi.genomic.fna readsONananas_unmapped.fastq  > readsONananasunmapped_ON_fungi.sam

In [ ]:
[comte@node30 ananas]$ samtools flagstat readsONananasunmapped_ON_fungi.sam 
2139434 + 0 in total (QC-passed reads + QC-failed reads)
976378 + 0 secondary
15312 + 0 supplementary
0 + 0 duplicates
1733344 + 0 mapped (81.02% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)

In [ ]:
# séparation des reads mappés et unmappés

# reads fungi
samtools view -@ 4 -bh -F 4 readsONananasunmapped_ON_fungi.sam > readsONananasunmapped_ON_fungi_mapped.sam 

# reads sans ananas et sans fungi
samtools view -@ 4 -bh -f 4 readsONananasunmapped_ON_fungi.sam > readsONananasunmapped_ON_fungi_unmapped.sam 

In [ ]:
# extraction des reads des deux bam

samtools fastq readsONananasunmapped_ON_fungi_unmapped.sam > readsONananasunmapped_ON_fungi_unmapped.fastq
samtools fastq readsONananasunmapped_ON_fungi_mapped.sam > readsONananasunmapped_ON_fungi_mapped.fastq

In [ ]:
# calcul du nombre de reads sans ananas et sans fungi

awk '{s++}END{print s/4}' readsONananasunmapped_ON_fungi_unmapped.fastq
406090


### mapping sur bactérie ? 

In [ ]:
# on peut faire la même chose sur les bactéries (mapping + garder que les unmappé sur les bactéries)
# Dans ce cas là, la bdd est ici:

wget -r --no-parent -A *genomic.fna.gz https://ftp.ncbi.nlm.nih.gov/refseq/release/bacteria/

# Cette base est si grosse que nous avons choisi de ne pas le faire.

### ajout reads ananas

In [ ]:
# Echantillonnage de 500 000 reads d'ananas

module load bioinfo/seqtk/1.3-r106
seqtk sample -s100 readsONananas_mapped.fastq 500000 > sub_readsananas.fastq

In [ ]:
# fusion de ces reads ananas avec les reads sans ananas/fungi

cat readsONananasunmapped_ON_fungi_unmapped.fastq sub_readsananas.fastq > test1.fastq

awk '{s++}END{print s/4}' test1.fastq 
906090
